# 🤖 Train SO-101 Robot with LeRobot

This notebook trains an **ACT (Action Chunking Transformer)** policy on your demonstration data from RoboSim.

## Prerequisites
- Dataset uploaded to HuggingFace Hub from RoboSim
- Google account (for Colab GPU access)

## Instructions
1. Enter your dataset ID below
2. Click **Runtime → Run all**
3. Wait ~2 hours for training
4. Download your trained model!

---

## Step 1: Configuration

Enter your HuggingFace dataset ID and token below:

In [ ]:
#@title Configuration { display-mode: "form" }
#@markdown ### Your Dataset
DATASET_REPO_ID = "your-username/your-dataset-name" #@param {type:"string"}

#@markdown ### HuggingFace Token (for saving trained model)
HF_TOKEN = "" #@param {type:"string"}

#@markdown ### Training Settings
TRAINING_STEPS = 50000 #@param {type:"integer"}
BATCH_SIZE = 8 #@param {type:"integer"}
SAVE_MODEL_TO_HF = True #@param {type:"boolean"}

#@markdown ---

# Validate inputs
if DATASET_REPO_ID == "your-username/your-dataset-name":
    print("⚠️  Please enter your actual dataset ID above!")
    print("   Example: hshadab/cube-pickup-training-1234567890")
else:
    print(f"✅ Dataset: {DATASET_REPO_ID}")
    print(f"✅ Training steps: {TRAINING_STEPS:,}")
    print(f"✅ Batch size: {BATCH_SIZE}")
    if SAVE_MODEL_TO_HF and HF_TOKEN:
        print(f"✅ Will save trained model to HuggingFace")
    elif SAVE_MODEL_TO_HF:
        print("⚠️  Enter HF_TOKEN to save model to HuggingFace")

## Step 2: Check GPU & Install Dependencies

In [ ]:
#@title Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"✅ Memory: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU detected!")
    print("   Go to Runtime → Change runtime type → GPU")

In [ ]:
#@title Install LeRobot (takes ~3 minutes)
print("Installing LeRobot...")
!pip install -q lerobot

# Verify installation
import lerobot
print(f"✅ LeRobot {lerobot.__version__} installed")

## Step 3: Load & Inspect Dataset

In [ ]:
#@title Load dataset from HuggingFace
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

print(f"Loading dataset: {DATASET_REPO_ID}")
dataset = LeRobotDataset(DATASET_REPO_ID)

print(f"\n✅ Dataset loaded!")
print(f"   Episodes: {dataset.num_episodes}")
print(f"   Total frames: {len(dataset)}")
print(f"   Features: {list(dataset.features.keys())}")

In [ ]:
#@title Preview a sample from the dataset
import matplotlib.pyplot as plt

# Get first frame
sample = dataset[0]

print("Sample data:")
for key, value in sample.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    else:
        print(f"  {key}: {value}")

## Step 4: Train ACT Policy

This will take **~2 hours** on a T4 GPU for 50,000 steps.

You can monitor progress in the output below.

In [ ]:
#@title Start training
import os

# Set HuggingFace token for model upload
if HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN

# Create output directory
OUTPUT_DIR = f"outputs/train/{DATASET_REPO_ID.split('/')[-1]}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"🚀 Starting training...")
print(f"   Dataset: {DATASET_REPO_ID}")
print(f"   Steps: {TRAINING_STEPS:,}")
print(f"   Output: {OUTPUT_DIR}")
print(f"\nThis will take ~2 hours. Go grab a coffee! ☕\n")

# Run training
!python -m lerobot.scripts.train \
    policy=act \
    env=so100_real \
    dataset_repo_id={DATASET_REPO_ID} \
    training.num_workers=4 \
    training.batch_size={BATCH_SIZE} \
    training.steps={TRAINING_STEPS} \
    training.save_freq=10000 \
    training.log_freq=100 \
    eval.n_episodes=0 \
    wandb.enable=false \
    hydra.run.dir={OUTPUT_DIR}

## Step 5: Save Trained Model

In [ ]:
#@title Upload trained model to HuggingFace
from huggingface_hub import HfApi, upload_folder
import os

if SAVE_MODEL_TO_HF and HF_TOKEN:
    # Find the checkpoint directory
    checkpoint_dir = f"{OUTPUT_DIR}/checkpoints/last/pretrained_model"
    
    if os.path.exists(checkpoint_dir):
        # Create model repo name
        dataset_name = DATASET_REPO_ID.split('/')[-1]
        model_repo_id = f"{DATASET_REPO_ID.split('/')[0]}/{dataset_name}-act-policy"
        
        print(f"Uploading model to: {model_repo_id}")
        
        api = HfApi(token=HF_TOKEN)
        
        # Create repo if it doesn't exist
        try:
            api.create_repo(model_repo_id, exist_ok=True)
        except Exception as e:
            print(f"Note: {e}")
        
        # Upload
        upload_folder(
            folder_path=checkpoint_dir,
            repo_id=model_repo_id,
            token=HF_TOKEN
        )
        
        print(f"\n✅ Model uploaded!")
        print(f"   https://huggingface.co/{model_repo_id}")
    else:
        print(f"❌ Checkpoint not found at {checkpoint_dir}")
        print("   Make sure training completed successfully.")
else:
    print("⚠️  Skipping upload (no HF_TOKEN or SAVE_MODEL_TO_HF=False)")
    print(f"   Model saved locally at: {OUTPUT_DIR}/checkpoints/")

In [ ]:
#@title Download model to your computer
from google.colab import files
import shutil

checkpoint_dir = f"{OUTPUT_DIR}/checkpoints/last/pretrained_model"

if os.path.exists(checkpoint_dir):
    # Create zip file
    zip_name = f"{DATASET_REPO_ID.split('/')[-1]}_act_policy"
    shutil.make_archive(zip_name, 'zip', checkpoint_dir)
    
    print(f"📥 Downloading {zip_name}.zip...")
    files.download(f"{zip_name}.zip")
else:
    print(f"❌ No checkpoint found. Run training first.")

## Step 6: Deploy to Real SO-101

Once you have the trained model, run this on your computer with the SO-101 connected:

```bash
# Install LeRobot on your computer
pip install lerobot

# Run the trained policy
python -m lerobot.scripts.control_robot record \
    --robot-path lerobot/configs/robot/so100.yaml \
    --policy-path PATH_TO_YOUR_MODEL \
    --fps 30
```

Or if you uploaded to HuggingFace:

```bash
python -m lerobot.scripts.control_robot record \
    --robot-path lerobot/configs/robot/so100.yaml \
    --policy.path=YOUR_USERNAME/YOUR_MODEL_REPO \
    --fps 30
```

---

## 🎉 Done!

You've successfully trained an ACT policy for your SO-101 robot.

**Next steps:**
1. Download or use the HuggingFace model
2. Connect your real SO-101 robot
3. Run the policy and watch it pick up objects!

**Need help?** Check out:
- [LeRobot Documentation](https://github.com/huggingface/lerobot)
- [RoboSim GitHub](https://github.com/YOUR_REPO)

---
*Generated by RoboSim - Train robots with AI*